<div>
<b><br><font face = "Gill Sans" size = "4"><center>Heart attack prediction with logistic regression</center></font></b>
</div>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [3]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,286 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.8 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,654 kB]

<span style="font-family:Gill Sans">
<h1>Hearth attach prediction with logisic regression</h1></span>

Let us use PySpark to predict if a person can suffer a heart attack based on some medical information.

Download the dataset from [Kaggle](https://www.kaggle.com/datasets/dileep070/heart-disease-prediction-using-logistic-regression?resource=download)

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, FloatType, IntegerType

# Create Spark Session in localhost
spark = SparkSession.builder.master("local").\
    appName("ITESO-LogisticRegression").\
    getOrCreate()

# Set spark context
sc = spark.sparkContext
sc.setLogLevel("ERROR")

heart_prediction_schema = StructType([
    StructField("male", IntegerType(), True),
    StructField("age", IntegerType(), True),
    StructField("education", IntegerType(), True),
    StructField("currentSmoker", IntegerType(), True),
    StructField("cigsPerDay", IntegerType(), True),
    StructField("BPMeds", IntegerType(), True),
    StructField("prevalentStroke", IntegerType(), True),
    StructField("prevalentHyp", IntegerType(), True),
    StructField("diabetes", IntegerType(), True),
    StructField("totChol", FloatType(), True),
    StructField("sysBP", FloatType(), True),
    StructField("diaBP", FloatType(), True),
    StructField("BMI", FloatType(), True),
    StructField("heartRate", IntegerType(), True),
    StructField("glucose", IntegerType(), True),
    StructField("TenYearCHD", IntegerType(), True)
])

heart_df = spark.read.format("csv").\
    option("header", "true").\
    option("mode", "permissive").\
    option("path", "/content/sample_data/framingham.csv").\
    schema(heart_prediction_schema).\
    load()
heart_df.printSchema()
heart_df.show(n=10, truncate=False)

root
 |-- male: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- education: integer (nullable = true)
 |-- currentSmoker: integer (nullable = true)
 |-- cigsPerDay: integer (nullable = true)
 |-- BPMeds: integer (nullable = true)
 |-- prevalentStroke: integer (nullable = true)
 |-- prevalentHyp: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- totChol: float (nullable = true)
 |-- sysBP: float (nullable = true)
 |-- diaBP: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- heartRate: integer (nullable = true)
 |-- glucose: integer (nullable = true)
 |-- TenYearCHD: integer (nullable = true)

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI  |heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+----

In [5]:
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
heart_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in heart_df.columns]).show()

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|   0|  0|      105|            0|        29|    53|              0|           0|       0|     50|    0|    0| 19|        1|    388|         0|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+



In [12]:
# Remove rows with at least one value as null
heart_df = heart_df.dropna()

In [13]:
heart_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in heart_df.columns]).show()

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|   0|  0|        0|            0|         0|     0|              0|           0|       0|      0|    0|    0|  0|        0|      0|         0|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+



In [33]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

# Rename the label column to "label"
heart_df = heart_df.withColumnRenamed("TenYearCHD", "label")

# Assemble the features into a single vector column
assembler = VectorAssembler(inputCols=[
    "male", "age", "education", "currentSmoker", "cigsPerDay", "BPMeds",
    "prevalentStroke", "prevalentHyp", "diabetes", "totChol", "sysBP",
    "diaBP", "BMI", "heartRate", "glucose"
], outputCol="features")

# Transform the data to include the features column
data_with_features = assembler.transform(heart_df).select("label", "features")

# Split the data into training and test sets
train_data, test_data = data_with_features.randomSplit([0.7, 0.3], seed=42)

# Create and fit the logistic regression model
lr = LogisticRegression(maxIter=10, regParam=0.01)
lr_model = lr.fit(train_data)

# Make predictions on the test data
predictions = lr_model.transform(test_data)


In [36]:
# Print coefficients
print("Coefficients: ", lr_model.coefficients)
print("Intercept: ", lr_model.intercept)
# Make predictions on the test set
predictions = lr_model.transform(test_data)

# Show predictions with label, prediction, and probability
predictions.select("label", "prediction", "probability").show(20, truncate=False)


Coefficients:  [0.4735396025580804,0.06041851836266556,-0.03971877928307416,0.2327369515049285,0.011143608436434018,0.4403698648334802,0.9518941968506953,0.17819944273141272,-0.0599048894289396,0.001665153192667679,0.012546906571086345,0.003169244721366329,0.006568041721294876,-0.0017887287753067774,0.00761459674519659]
Intercept:  -8.3516206112187
+-----+----------+-----------------------------------------+
|label|prediction|probability                              |
+-----+----------+-----------------------------------------+
|0    |0.0       |[0.9787648567524393,0.021235143247560706]|
|0    |0.0       |[0.9768582477203768,0.02314175227962323] |
|0    |0.0       |[0.977182692308456,0.022817307691543998] |
|0    |0.0       |[0.9818163357412204,0.018183664258779597]|
|0    |0.0       |[0.9769288665751202,0.023071133424879764]|
|0    |0.0       |[0.976143718517526,0.023856281482473984] |
|0    |0.0       |[0.9775945400869978,0.022405459913002246]|
|0    |0.0       |[0.9782016359655559,0